In [41]:
import numpy as np
import pandas as pd
import taxonomy as t
import re
from bitstring import BitArray, BitStream

paf="contig.paf"
t.loadTaxonomy( "database" )

In [43]:
# reading PAF file and dropping columns after s1. Indexing contig name for quick access.

df = pd.read_csv(paf,
    sep='\t',
    header=None,
    names=['qname','qlen','qstart','qend','strand','tname','tlen','tstart','tend','match_bp','mapping_bp','mqua','tp','cm','score','opt1','opt2'],
    index_col=['qname'],
    usecols=['qname','qlen','qstart','qend','strand','tname','tlen','tstart','tend','match_bp','mapping_bp','score'],
)

df['score'] = df['score'].str.replace('s1:i:','').astype(int)
df['ctg'] = df.index

In [44]:
df.index.unique()

Index(['11153_001_HostRemoved_074501', '11153_001_HostRemoved_107444',
       '11153_001_HostRemoved_029610', '11153_001_HostRemoved_075706',
       '11153_001_HostRemoved_046140', '11153_001_HostRemoved_043003',
       '11153_001_HostRemoved_037615', '11153_001_HostRemoved_101400',
       '11153_001_HostRemoved_056945', '11153_001_HostRemoved_041076',
       ...
       '11153_001_HostRemoved_103500', '11153_001_HostRemoved_031612',
       '11153_001_HostRemoved_014908', '11153_001_HostRemoved_067848',
       '11153_001_HostRemoved_019428', '11153_001_HostRemoved_028855',
       '11153_001_HostRemoved_020332', '11153_001_HostRemoved_052392',
       '11153_001_HostRemoved_084248', '11153_001_HostRemoved_038702'],
      dtype='object', name='qname', length=3707)

In [45]:
print(len(df))

df.loc['11153_001_HostRemoved_074501']

20001


,qlen,qstart,qend,strand,tname,tlen,tstart,tend,match_bp,mapping_bp,score,ctg
qname,,,,,,,,,,,,
11153_001_HostRemoved_074501,939,16,895,+,NC_021252.1,8948591,7603144,7604005,144,879,140,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,16,895,-,NZ_CP008953.1,8961318,1017093,1017954,144,879,140,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,16,895,+,NC_018266.1,10246920,8793356,8794217,139,879,135,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,16,895,+,NC_022116.1,10246864,8793300,8794161,139,879,135,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,16,895,+,NC_017186.1,10236779,8783442,8784303,139,879,135,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,16,895,+,NC_014318.1,10236715,8783383,8784244,139,879,135,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,7,887,+,NZ_CP009110.1,7237391,5968403,5969268,119,880,116,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,7,895,+,NC_013093.1,8248144,7392108,7392981,113,888,110,11153_001_HostRemoved_074501
11153_001_HostRemoved_074501,939,7,895,+,NZ_CP023445.1,8131572,7259170,7260043,113,888,110,11153_001_HostRemoved_074501


In [61]:
def lineageLCR(taxids):
        """ lineageLCR
        Find late common rank (LCR) from two lineage
        """
        ranks = ['strain','species','genus','family','order','class','phylum','superkingdom']

        merged_dict = t._autoVivification()
        for tid in taxids:
                lng = t.taxid2lineageDICT(tid, 1, 1)
                for r in lng:
                        ttid = lng[r]['taxid']
                        if ttid in merged_dict[r]:
                                merged_dict[r][ttid] += 1
                        else:   
                                merged_dict[r][ttid] = 1

        for r in ranks:
                if len(merged_dict[r]) == 1:
                        for ttid in merged_dict[r]:
                                # skip if no tid in this rank
                                if ttid==0:
                                        continue
                                tname = t.taxid2name(ttid)
                                return r

        return "root"

In [71]:
# sorting by contig, score, then qstart and qend
df.sort_values(by=['ctg', 'score', 'qstart', 'qend'], ascending=False, inplace=True)

# only keep rows with max score for the same mapped regions
df['score_max'] = df.groupby(['ctg','qstart','qend'])['score'].transform(max)
df = df[ df['score']==df['score_max'] ]

df['taxid'] = df['tname'].apply(t.acc2taxid)
df['region'] = np.nan

df.loc['11153_001_HostRemoved_074501'].groupby(['ctg','qstart','qend'])

mapping_bp                            \
                                              count   mean  std    min    25%   
ctg                          qstart qend                                        
11153_001_HostRemoved_074501 7      887         1.0  880.0  NaN  880.0  880.0   
                                    895         2.0  888.0  0.0  888.0  888.0   
                             16     895         2.0  879.0  0.0  879.0  879.0   

                                                              match_bp         \
                                            50%    75%    max    count   mean   
ctg                          qstart qend                                        
11153_001_HostRemoved_074501 7      887   880.0  880.0  880.0      1.0  119.0   
                                    895   888.0  888.0  888.0      2.0  113.0   
                             16     895   879.0  879.0  879.0      2.0  144.0   

                                            ...            tlen             \
                                            ...             75%        max   
ctg                          qstart qend    ...                              
11153_001_HostRemoved_074501 7      887     ...      7237391.00  7237391.0   
                                    895     ...      8219001.00  8248144.0   
                             16     895     ...      8958136.25  8961318.0   

                                         tstart                           \
                                          count       mean           std   
ctg                          qstart qend                                   
11153_001_HostRemoved_074501 7      887     1.0  5968403.0           NaN   
                                    895     2.0  7325639.0  9.400136e+04   
                             16     895     2.0  4310118.5  4.657041e+06   

                                                                            \
                                                min         25%        50%   
ctg                          qstart qend                                     
11153_001_HostRemoved_074501 7      887   5968403.0  5968403.00  5968403.0   
                                    895   7259170.0  7292404.50  7325639.0   
                             16     895   1017093.0  2663605.75  4310118.5   

                                                                 
                                                 75%        max  
ctg                          qstart qend                         
11153_001_HostRemoved_074501 7      887   5968403.00  5968403.0  
                                    895   7358873.50  7392108.0  
                             16     895   5956631.25  7603144.0  

[3 rows x 72 columns]

In [12]:
def get_extra_regions(mask, qstart, qend, qlen):
    # init bitstring "add" if it's empty
    if not add:
        add = BitArray("0b%s%s%s"%("0"*qstart, "1"*(qend-qstart+1), "0"*(qlen-qend-1)))
    add_mask = add&(mask^add)
    mask = mask|add
    
    p = re.compile('1+')
    iterator = p.finditer(add_mask.bin)
    
    return (mask, [match.span() for match in iterator])

def aggregate_ctg(ctg_df):
    """
    aggregate alignments
    """
    ranks = ["superkingdom","phylum","class","order","family","genus","species","strain"]
    ctg_mask = BitArray("0b"+"0"*ctg_df.iloc[1].qlen.item())
    ctg = t._autoVivification()
    
    # calculate aggregate region for each contig
    for i in range(1, len(ctg_df)+1):
        tname   = ctg_df.iloc[i].tname.item()
        taxid   = t.acc2taxid(tname)
        lineage = t.taxid2lineageDICT(taxid, 1, 1)

        (qstart, qend, qlen) = ctg_df.iloc[i][['qstart','qend','qlen']].tolist()
        
        if not ctg[qstart][qend]:
            (ctg_mask, region) = get_extra_regions(ctg_mask, qstart, qend, qlen)
        else:
            
        
        if region:
            ctg_df.iloc[i]['region'] = region
    
        # calculate ranks
        for rank in ranks:
            if not rank in p:
                p[rank]={}
    


    if name in p[rank]:
        pcov = p[rank][name]['pcov']
    else:
        pcov = Bits("0b%s"%"0"*qlen)
        p[rank][name]={}
        p[rank][name]['pcov']        = pcov
        p[rank][name]['TOL_HIT_LEN'] = 0
        p[rank][name]['NUM_HIT']     = 0
        p[rank][name]['AVG_IDT']     = 0
        p[rank][name]['TOL_MISM']    = []

    mask = Bits("0b%s%s%s" % ("0"*(qstart-1), "1"*mapping_bp, "0"*(qlen-qend)))

    p[rank][name]['pcov']         = pcov | mask
    p[rank][name]['TOL_HIT_LEN'] += mapping_bp # total mapped bp
    p[rank][name]['NUM_HIT']     += 1 # number of hits
    p[rank][name]['TOL_MISM']    += mapping_bp - match_bp # total mismatches
    p[rank][name]['AVG_IDT']      = (p[rank][name]['TOL_HIT_LEN'] - p[rank][name]['TOL_MISM'])/p[rank][name]['TOL_HIT_LEN']

    
            
genome_seq_cov(df.loc['11153_001_HostRemoved_074501'])

SyntaxError: invalid syntax (<ipython-input-12-00541aa71730>, line 65)

In [ ]:
def accCov(acc_cov,id,region):
    (qs, qe) = region

    while( $acc_cov =~ /0+/g ){
        my ($us, $ue) = ($-[0]+1,$+[0]);
        last if $us > $qe;
        if( $qs>=$us && $qe<=$ue){ #whole overlapping
            my $len = $qe-$qs+1;
            substr $acc_cov, $qs-1, $len, ${id}x$len;
        }
        elsif( $us>=$qs && $qe>=$us && $ue>=$qe ){ #cov overlapping 3" 0s
            my $len = $ue-$qs+1;
            substr $acc_cov, $qs-1, $len, ${id}x$len;
        }
        elsif( $qs>=$us && $qs<=$ue && $qe>$ue ){ #overlapping 5"
            my $len = $qe-$us+1;
            substr $acc_cov, $us-1, $len, ${id}x$len;
        }
    }

    return acc_cov

def accCovSummary:
    my ($acc_cov,$map) = @_;
    my $c;
    my $csum;
    while( $acc_cov =~ /(.)\1*/g ){
        my ($qs,$qe) = ($-[0]+1,$+[0]);
        my $tax = $map->{ord($1)};

        #dealing with an upper limit of '32766' on the MAX value of the regex {MIN,MAX} quantifier.
        my ($prev_end) = $csum->{$tax}[-1] =~ /\.\.(\d+)/;
        if( defined $prev_end && $prev_end+1 == $qs ){
            $csum->{$tax}[-1] =~ s/\.\.$prev_end/\.\.$qe/;
        }
        else{
            push @{$csum->{$tax}}, "$qs..$qe";
        }
    }
    return $csum;

NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1
NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1
NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1
NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1
NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1
NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1
NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1
NC_021252.1
NZ_CP008953.1
NC_018266.1
NC_022116.1
NC_017186.1
NC_014318.1
NZ_CP009110.1
NC_013093.1
NZ_CP023445.1


In [ ]:

            foreach my $region ( keys %{$seq->{$pname}->{$taxid}} ){
                my ($qs, $qe) = $region =~ /^(\d+)\.\.(\d+)$/;
                my $end = length($pcov);
                my $nm = $seq->{$pname}->{$taxid}->{$region};
                
                #update linear length
                my $str = "0"x($qs-1) . "1"x($qe-$qs+1) . "0"x($end-$qe);
                $pcov = $pcov | $str;
                #total mapped
                $p->{$rank}->{$name}->{TOL_HIT_LEN} ||= 0;
                $p->{$rank}->{$name}->{TOL_HIT_LEN} += $qe-$qs+1;
                #number of hits
                $p->{$rank}->{$name}->{NUM_HIT} ||= 0;
                $p->{$rank}->{$name}->{NUM_HIT}++;

                #distance
                $p->{$rank}->{$name}->{TOL_MISM} ||= 0;
                $p->{$rank}->{$name}->{TOL_MISM} += $nm;
            }
            $p->{$rank}->{$name}->{AVG_IDT} = ($p->{$rank}->{$name}->{TOL_HIT_LEN} - $p->{$rank}->{$name}->{TOL_MISM})/$p->{$rank}->{$name}->{TOL_HIT_LEN};
            $p->{$rank}->{$name}->{LINEAR_LEN} = $pcov;
        }

        # 
        foreach my $name ( keys %{$p->{$rank}} ){
            $p->{$rank}->{$name}->{LINEAR_LEN} =~ s/0//g;
            my $sum = length($p->{$rank}->{$name}->{LINEAR_LEN});

            $p->{$rank}->{$name}->{LINEAR_LEN} = $sum;
            $r->{$rank}->{TOL_LINEAR_LEN} ||= 0;
            $r->{$rank}->{TOL_LINEAR_LEN} += $sum;
        }

        #accumulated coverage
        my $acc_cov;
        $acc_cov = "0"x$length->{$pname};
        my $map;
        my $map->{48}="unclassified";
        my $mid_ascii = 49;

        foreach my $cnt ( sort {$a<=>$b} keys %{$cov->{$pname}} )
        {
            foreach my $taxid ( keys %{$cov->{$pname}->{$cnt}} )
            {
                my $name = taxid2rank($taxid, $rank);    
                #upper taxa
                my $upname = taxid2rank($taxid, $upper_level);
                $upname = "NA" unless $upname;
                $name = "$upname $rank" unless $name;

                unless( defined $map->{$name} ){
                    $map->{$name} = $mid_ascii;
                    $map->{$mid_ascii} = $name;
                    $mid_ascii++;
                }

                my $region = $cov->{$pname}->{$cnt}->{$taxid};
                $acc_cov = &accCov($acc_cov, chr($map->{$name}), $region);
            }
        }
        my $csum = &accCovSummary($acc_cov, $map);
        foreach my $name ( keys %$csum )
        {
            $p->{$rank}->{$name}->{ACC_COV_RGN} = join ";", @{$csum->{$name}};
            my $len=0;
            foreach my $rgn ( @{$csum->{$name}} )
            {
                my ($qs,$qe) = $rgn =~ /(\d+)\.\.(\d+)/;
                $len += $qe-$qs+1;
            }
            $p->{$rank}->{$name}->{ACC_COV_LEN} = $len;
        }

        $upper_level = $rank;
    }